# Introduction to Jupyter (JUlia, PYThon, and R), bash, and CUDA

### We can do both text and code in Jupyter, but first let's start with some bash

In [1]:
!ls

cpuAdd.c	 numbapro-examples
CUDAkernels.cu	 Tutorial1-Introduction-to-Numba.ipynb
CUDATools.py	 Tutorial1-NumbaProNbody.ipynb
gpuAdd.cu	 Tutorial2-CUDA_Basics_PyCUDA.ipynb
gpu_prof.py	 Tutorial2-CUDAMemories.ipynb
images		 Tutorial2-Scikit-CUDA-example.ipynb
job.jupyter.gpu


In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


### Jupyter has some "magic" builtin commands. Here's how to list them

In [3]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autoawait  %autocall  %automagic  %autosave  %bookmark  %cat  %cd  %clear  %colors  %config  %connect_info  %cp  %debug  %dhist  %dirs  %doctest_mode  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %lf  %lk  %ll  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %lx  %macro  %magic  %man  %matplotlib  %mkdir  %more  %mv  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %popd  %pprint  %precision  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %rep  %rerun  %reset  %reset_selective  %rm  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %%markdown  %%perl  %%prun  %%pypy  %%python  %%pyth

And now let's use the bash cell magic

In [4]:
%%bash
echo "Hello 1"
sleep 5
echo "Hello 2"

Hello 1
Hello 2


Here's how to get some documentation for the magics

In [5]:
%autosave?

In [6]:
%autosave 120

Autosaving every 120 seconds


And the "magic" source code

In [7]:
%autosave??

And here's how we get some documentation on Python functions.

In [8]:
?str.replace

Or on important Python packages such as numpy

In [9]:
import numpy
?numpy

One can list the currently set environment

In [10]:
%env

{'EBVERSIONLIBSODIUM': '1.0.16',
 'EBVERSIONIRKERNEL': '1.0.2',
 'MODULES_PREFIX': 'SARA',
 'JUPYTERHUB_CLIENT_ID': 'jupyterhub-user-valeriuc',
 'LD_LIBRARY_PATH': '/sw/arch/Debian9/EB_production/2019/software/cuDNN/7.6.3-CUDA-10.0.130/lib64:/sw/arch/Debian9/EB_production/2019/software/CUDA/10.0.130/nvvm/lib64:/sw/arch/Debian9/EB_production/2019/software/CUDA/10.0.130/extras/CUPTI/lib64:/sw/arch/Debian9/EB_production/2019/software/CUDA/10.0.130/lib64:/sw/arch/Debian9/EB_production/2019/software/R/3.5.1-foss-2018b/lib64/R/lib:/sw/arch/Debian9/EB_production/2019/software/R/3.5.1-foss-2018b/lib64:/sw/arch/Debian9/EB_production/2019/software/GSL/2.5-GCC-7.3.0-2.30/lib:/sw/arch/Debian9/EB_production/2019/software/UDUNITS/2.2.26-foss-2018b/lib:/sw/arch/Debian9/EB_production/2019/software/ICU/61.1-GCCcore-7.3.0/lib:/sw/arch/Debian9/EB_production/2019/software/libsndfile/1.0.28-GCCcore-7.3.0/lib:/sw/arch/Debian9/EB_production/2019/software/NLopt/2.4.2-GCCcore-7.3.0/lib:/sw/arch/Debian9/EB_prod

Or modify it

In [11]:
%env OMP_NUM_THREADS=16

env: OMP_NUM_THREADS=16


That's how we write a Python program

In [5]:
%%writefile hello_world.py
if __name__ == "__main__":
	print("Hello World!")

Writing hello_world.py


And that's how we run it

In [6]:
%run ./hello_world.py

Hello World!


And some bash magic

In [14]:
%%bash
for i in a b c;
do
echo $i
done

a
b
c


# Introduction to Python GPU Programming with Numba

In [15]:
%env SURFSARA_LIBRARY_PATH ""
%env SURFSARA_INCLUDE_PATH ""
!pip install numba --user

env: SURFSARA_LIBRARY_PATH=""
env: SURFSARA_INCLUDE_PATH=""


In [16]:
# Misc. import
from IPython.display import Image 
%matplotlib inline

# Numba
* Opensource BSD license
* Basic CUDA GPU JIT compilation
* OpenCL support coming

In [17]:
import numba
print("numba", numba.__version__)

numba 0.48.0


# The CUDA GPU

- A massively parallel processor (many cores)
    - 100 threads, 1,000 threads, and more
- optimized for data throughput
    - Simple (shallow) cache hierarchy (improving with each generation)
    - GPUs feature "shared memory" that is explicitly addressable
- CPU is latency optimized
    - Deep cache hierarchy
    - L1, L2, L3 caches
- GPU execution model is different
- GPU forces you to think and program *in parallel*

In [18]:
# Get all the imports we need
import numba.cuda
import numpy as np
import math

my_gpu = numba.cuda.get_current_device()
print("Running on GPU:", my_gpu.name)
cores_per_capability = {
    1: 8,
    2: 32,
    3: 192,
    5: 128,
    6: 128
}
cc = my_gpu.compute_capability
print("Compute capability: ", "%d.%d" % cc, "(Numba requires >= 2.0)")
majorcc = cc[0]
print("Number of streaming multiprocessor:", my_gpu.MULTIPROCESSOR_COUNT)
cores_per_multiprocessor = cores_per_capability[majorcc]
print("Number of cores per mutliprocessor:", cores_per_multiprocessor)
total_cores = cores_per_multiprocessor * my_gpu.MULTIPROCESSOR_COUNT
print("Number of cores on GPU:", total_cores)

Running on GPU: b'GeForce GTX 1080 Ti'
Compute capability:  6.1 (Numba requires >= 2.0)
Number of streaming multiprocessor: 28
Number of cores per mutliprocessor: 128
Number of cores on GPU: 3584


# High-level Array-Oriented Style

- Use NumPy array as a unit of computation
- Use NumPy universal function (ufunc) as an abstraction of computation and scheduling
- ufuncs are elementwise functions
- If you use NumPy, you are using ufuncs

In [ ]:
print(np.sin, "is of type", type(np.sin))
print(np.add, "is of type", type(np.add))

<ufunc 'sin'> is of type <class 'numpy.ufunc'>
<ufunc 'add'> is of type <class 'numpy.ufunc'>


# Vectorize

- generate a ufunc from a python function
- converts scalar function to elementwise array function
- Numba provides CPU and GPU support

In [ ]:
# CPU version
@numba.vectorize(['float32(float32, float32)',
                  'float64(float64, float64)'], target='cpu')
def cpu_sincos(x, y):
    return math.sin(x) * math.cos(y)

# CUDA version
@numba.vectorize(['float32(float32, float32)',
                     'float64(float64, float64)'], target='cuda')
def gpu_sincos(x, y):
    return math.sin(x) * math.cos(y)

```
@numba.vectorize(<list of signatures>, target=<'cpu', 'cuda'>)
```

- A ufunc can be overloaded to work on multiple type signatures


### Test it out

- 2 input arrays
- 1 output array
- 1 million doubles (8 MB) per array
- Total 24 MB of data

In [ ]:
# Generate data
n = 1000000
x = np.linspace(0, np.pi, n)
y = np.linspace(0, np.pi, n)

# Check result
np_ans = np.sin(x) * np.cos(y)
nb_cpu_ans = cpu_sincos(x, y)
nb_gpu_ans = gpu_sincos(x, y)

print("CPU vectorize correct: ", np.allclose(nb_cpu_ans, np_ans))
print("GPU vectorize correct: ", np.allclose(nb_gpu_ans, np_ans))

CPU vectorize correct:  True
GPU vectorize correct:  True


## Benchmark

In [ ]:
print("NumPy")
%timeit np.sin(x) * np.cos(y)

print("CPU vectorize")
%timeit cpu_sincos(x, y)

print("GPU vectorize")
%timeit gpu_sincos(x, y)

# Optional cleanup 
del x, y

NumPy
83.9 ms ± 147 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
CPU vectorize
109 ms ± 24.2 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
GPU vectorize
10.6 ms ± 20.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


- CPU vectorize time is similar to pure NumPy time because ``sin()`` and ``cos()`` calls dominate the time.
- GPU vectorize is a lot faster



## Behind the scence




### Automatic memory transfer

- NumPy arrays are automatically transferred
    - CPU -> GPU
    - GPU -> CPU


### Automatic work scheduling

- The work is distributed the across all threads on the GPU
- The GPU hardware handles the scheduling



### Automatic GPU memory management

- GPU memory is tied to object lifetime
- freed automatically

# Generalized Universal Function (guvectorize)

- Vectorize is limited to scalar arguments in the core function
- GUVectorize accepts array arguments

In [ ]:
@numba.guvectorize(['void(float32[:,:], float32[:,:], float32[:,:])'],
                      '(m, n),(n, p)->(m, p)', target='cuda')
def batch_matrix_mult(a, b, c):
    for i in range(c.shape[0]):
        for j in range(c.shape[1]):
            tmp = 0
            for n in range(a.shape[1]):
                 tmp += a[i, n] * b[n, j]
            c[i, j] = tmp

```
@numba.guvectorize(<list of function signatures>, <a string to desc the shape signature>, target=<'cpu', 'cuda'>)
```

In [ ]:
a = np.arange(1.0, 10.0, dtype=np.float32).reshape(3,3)
b = np.arange(1.0, 10.0, dtype=np.float32).reshape(3,3)

# Use the builtin matrix_multiply in NumPy for CPU test
import numpy.core.umath_tests as ut

# Check result
print('NumPy result')
np_ans = ut.matrix_multiply(a, b)
print(np_ans)

print('Numba GPU result')
gpu_ans = batch_matrix_mult(a, b)
print(gpu_ans)

assert np.allclose(np_ans, gpu_ans)


NumPy result
[[ 30.  36.  42.]
 [ 66.  81.  96.]
 [102. 126. 150.]]
Numba GPU result
[[ 30.  36.  42.]
 [ 66.  81.  96.]
 [102. 126. 150.]]


/sw/arch/Debian9/EB_production/2019/software/IPython/7.2.0-foss-2018b-Python-3.6.6/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  """


### Test it out

- batch multiply 4 million 2x2 matrices 

In [ ]:
n = 4000000
dim = 2
a = np.random.random(n * dim * dim).astype(np.float32).reshape(n, dim, dim)
b = np.random.random(n * dim * dim).astype(np.float32).reshape(n, dim, dim)

print('NumPy time')
%timeit ut.matrix_multiply(a, b)

print('Numba GPU time')
%timeit batch_matrix_mult(a, b)

NumPy time
177 ms ± 2.58 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Numba GPU time
115 ms ± 5.09 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


- GPU time seems to be similar to CPU time
- Let's try to profile the code

In [ ]:
!cat gpu_prof.py

import numba
import numpy as np
import numpy.core.umath_tests as ut
import timeit
@numba.guvectorize(['void(float32[:,:], float32[:,:], float32[:,:])'],
                      '(m, n),(n, p)->(m, p)', target='cuda')
def batch_matrix_mult(a, b, c):
    for i in range(c.shape[0]):
        for j in range(c.shape[1]):
            tmp = 0
            for n in range(a.shape[1]):
                 tmp += a[i, n] * b[n, j]
            c[i, j] = tmp


n = 4000000
dim = 2
a = np.random.random(n * dim * dim).astype(np.float32).reshape(n, dim, dim)
b = np.random.random(n * dim * dim).astype(np.float32).reshape(n, dim, dim)

testcode_gpu = ''' 
def test_gpu(): 
    return batch_matrix_mult(a,b)
'''

print('Numba GPU time')
starttime = timeit.default_timer()
print("The start time is :",starttime)
for i in range(7):
    gpu_ans = batch_matrix_mult(a,b)
print("The time difference is :", timeit.default_timer() - starttime)


In [ ]:
!nvprof python gpu_prof.py

==23274== NVPROF is profiling process 23274, command: python gpu_prof.py
Numba GPU time
The start time is : 2798465.449988605
The time difference is : 0.7944908211939037
==23274== Profiling application: python gpu_prof.py
==23274== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   64.98%  459.17ms         7  65.596ms  65.247ms  66.481ms  [CUDA memcpy DtoH]
                   33.78%  238.67ms        14  17.048ms  16.424ms  20.428ms  [CUDA memcpy HtoD]
                    1.25%  8.7997ms         7  1.2571ms  1.2520ms  1.2604ms  cudapy::__main__::__gufunc_batch_matrix_mult$242(Array<float, int=3, A, mutable, aligned>, Array<float, int=3, A, mutable, aligned>, Array<float, int=3, A, mutable, aligned>)
      API calls:   45.57%  475.55ms         7  67.936ms  67.446ms  68.719ms  cuMemcpyDtoH
                   26.10%  272.38ms         1  272.38ms  272.38ms  272.38ms  cuDevicePrimaryCtxRetain
                   23.18%  241.8


## Manually Transfer the data to the GPU

- This will let us see the actual compute time without the CPU<->GPU transfer

In [ ]:
dc = numba.cuda.device_array_like(a)
da = numba.cuda.to_device(a)
db = numba.cuda.to_device(b)

* ```numba.cuda.device_array_like``` allocate without initialization with the type and shape of another array.
    * similar to ```numpy.empty_like(a)```
* ```numba.cuda.to_device``` create a GPU copy of the CPU array



## Pure compute time on the GPU

In [ ]:
def check_pure_compute_time(da, db, dc):
    batch_matrix_mult(da, db, out=dc)
    numba.cuda.synchronize()   # ensure the call has completed
    
%timeit check_pure_compute_time(da, db, dc)
del da, db, dc

3.83 ms ± 9.12 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


* Actual compute time is **a lot faster**
* PCI-express transfer overhead

#### Tips
If you have a sequence of ufuncs to apply, pin the data on the GPU by manual transfer

-----

# Low-Level Approach: @numba.cuda.jit

- Numba can generate CUDA functions with the `@jit` decorator
- Decorated function follows CUDA execution model 

## CUDA Execution Model

- Kernel functions
    - visible to the host CPU
    - cannot return any value
        - use output argument
    - associates to a _grid_
- Grid
    - a group of blocks
    - 1D, 2D, 3D
- Blocks
    - a group of threads
    - 1D, 2D, 3D  
- Every thread executes the same kernel
    - thread can use the grid, block, thread coordinate system to determine its ID

In [ ]:
Image(url="https://docs.nvidia.com/cuda/cuda-c-programming-guide/graphics/grid-of-thread-blocks.png")

## Compiling a CUDA Kernel

In [ ]:
from numba import cuda

@numba.cuda.jit("void(float32[:], float32[:], float32[:])")
def vadd(arr_a, arr_b, arr_out):
    tx = cuda.threadIdx.x
    bx = cuda.blockIdx.x
    bw = cuda.blockDim.x    # number of threads per block
    i = tx + bx * bw
    if i >= arr_out.size:
        return
    arr_out[i] = arr_a[i] + arr_b[i]

### Code Explained

#### Define a CUDA kernel with three 1D float32 arrays as args

```
@numba.cuda.jit("void(float32[:], float32[:], float32[:])")
def vadd(arr_a, arr_b, arr_out):
```

#### Map thread, block coordinate to global position
```
    tx = cuda.threadIdx.x   # thread label (along x dimension)
    bx = cuda.blockIdx.x    # block label (along x dimension)
    bw = cuda.blockDim.x    # number of threads in each block (along x dimension)
    i = tx + bx * bw        # flattened linear address for each thread
```
or simplified to:
```
    i = cuda.grid(1)
``` 

#### Ensure global position is within array size

```
    if i >= arr_out.size:
        return
```

#### The actual work

```
    arr_out[i] = arr_a[i] + arr_b[i]
```

## Launch kernel

#### Prepare data

In [ ]:
n = 100
a = np.arange(n, dtype=np.float32)
b = np.arange(n, dtype=np.float32)
c = np.empty_like(a)                 # Must prepare the output array to hold the result


#### Calculate thread, block count

- thread count is set to **warp size** of the GPU
    - Warp size is similar to SIMD vector width on the CPU
    - **performance tips**: set thread count to multiple of warp size
- block count is ceil(n/thread_ct)

**Note:**
This will launch more threads than there are elements in the array

In [ ]:
thread_ct = my_gpu.WARP_SIZE
block_ct = int(math.ceil(float(n) / thread_ct))

print("Threads per block:", thread_ct)
print("Block per grid:", block_ct)

#### Launch kernel

Kernel function object uses the ``__getitem__`` (indexing notation) to configure the grid and block dimensions.

```
    kernel_function[griddim, blockdim](*args)
```

- **griddim**
    - Number of blocks per grid (grid dimension)
    - type: int for 1d or 1,2,3-tuple of ints for 1d, 2d, or 3d respectively

- **blockdim**: 
    - Number of threads per block (blockdim dimension)
    - type: int for 1d or 1,2,3-tuple of ints for 1d, 2d, or 3d respectively


In [ ]:
vadd[block_ct, thread_ct](a, b, c)    # Last argument is the output array in this case
print(c)

# Example: Matrix Matrix Multiplication

- Show manual caching with shared memory
- Not the best matrix matrix multiplication implementation

#### Prepare constants

In [ ]:
from numba import float32

bpg = 150
tpb = 32
n = bpg * tpb
shared_mem_size = (tpb, tpb)
griddim = bpg, bpg
blockdim = tpb, tpb

#### Naive version

In [ ]:
Image(url="https://docs.nvidia.com/cuda/cuda-c-programming-guide/graphics/matrix-multiplication-without-shared-memory.png")

In [ ]:
@numba.cuda.jit("void(float32[:,:], float32[:,:], float32[:,:])")
def naive_matrix_mult(A, B, C):
    x, y = cuda.grid(2)
    if x >= n or y >= n:
        return

    C[y, x] = 0
    for i in range(n):
        C[y, x] += A[y, i] * B[i, x]


#### Optimized version (shared memory + blocking)

In [ ]:
Image(url="https://docs.nvidia.com/cuda/cuda-c-programming-guide/graphics/matrix-multiplication-with-shared-memory.png")

In [ ]:
@numba.cuda.jit("void(float32[:,:], float32[:,:], float32[:,:])")
def optimized_matrix_mult(A, B, C):
    # Declare shared memory
    sA = cuda.shared.array(shape=shared_mem_size, dtype=float32)
    sB = cuda.shared.array(shape=shared_mem_size, dtype=float32)
    
    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    x, y = cuda.grid(2)

    acc = 0
    for i in range(bpg):
        if x < n and y < n:
            # Prefill cache
            sA[ty, tx] = A[y, tx + i * tpb]
            sB[ty, tx] = B[ty + i * tpb, x]

        # Synchronize all threads in the block
        cuda.syncthreads()

        if x < n and y < n:
            # Compute product
            for j in range(tpb):
                acc += sA[ty, j] * sB[j, tx]

        # Wait until all threads finish the computation
        cuda.syncthreads()

    if x < n and y < n:
        C[y, x] = acc

#### Prepare data

In [ ]:
# Prepare data on the CPU
A = np.array(np.random.random((n, n)), dtype=np.float32)
B = np.array(np.random.random((n, n)), dtype=np.float32)

print("N = %d x %d" % (n, n))

# Prepare data on the GPU
dA = cuda.to_device(A)
dB = cuda.to_device(B)
dC = cuda.device_array_like(A)

#### Benchmark

In [ ]:
# Time the naive version
import time
s = time.time()
naive_matrix_mult[griddim, blockdim](dA, dB, dC)
numba.cuda.synchronize()
e = time.time()
unopt_ans = dC.copy_to_host()
tcuda_unopt = e - s

# Time the optimized version
s = time.time()
optimized_matrix_mult[griddim, blockdim](dA, dB, dC)
numba.cuda.synchronize()
e = time.time()
opt_ans = dC.copy_to_host()
tcuda_opt = e - s

#### Result

In [ ]:
#assert np.allclose(unopt_ans, opt_ans)
print("Without shared memory:", "%.2f" % tcuda_unopt, "s")
print("With shared memory:", "%.2f" % tcuda_opt, "s")

# Summary

- Numba
    - opensource low-level GPU support
    - CUDA kernel ``@numba.cuda.jit``
    - vectorize ``@numba.vectorize``
    - guvectorize ``@numba.guvectorize``
    - profiling numba code
